In [ ]:
import pandas as pd
import statistics
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/abeam_final_project/cp_data_eng_full.csv') #import raw data


In [ ]:
null_table=(data.isna().sum()*100/data.shape[0]).sort_values(ascending=False).to_frame().reset_index()

null_table.rename({'index':'col',0:'null_percentage'},axis=1,inplace=True)

null_table

In [ ]:
#Percentage of missing value
null1=(data.isnull().sum()/len(data)*100).sort_index(ascending=True)

In [ ]:
#Duplicated Data Check
data.duplicated().sum()

In [ ]:
#Drop data who doesnt have response
data_non_na=data.dropna(subset='Response')

In [ ]:
#Percentage of missing value
null2=(data_non_na.isnull().sum()/len(data_non_na)*100).sort_values(ascending=False)

In [ ]:
null2

In [ ]:
null1-null2

# Univariate Analysis

## Card Information

In [ ]:
card_information=['Card_type','Shopping','Cashing','ETC','Card_contraction','Card_grade','expiration_date','shopping_avg','cashing_avg','ETC_avg','shopping_rec','cashing_rec','ETC_rec']

In [ ]:
#change the category in Card_grade column
print(data.Card_grade.value_counts().index  )

### Card grade

In [ ]:
#card grade
data['Card_grade']=data['Card_grade'].str.replace('platinum loyal card','loyal(platinum)card')
data['Card_grade']=data['Card_grade'].str.replace('loyal card','loyal(platinum)card')
data['Card_grade']=data['Card_grade'].str.replace('platinum card','loyal(platinum)card')

In [ ]:
def silver_god(x):
  if x=='silver card (classic)':
    return 'silver card'
  elif x=='gold card(superior)':
    return 'gold card'
  else:
    return x

In [ ]:
data['Card_grade']=data['Card_grade'].apply(silver_god) #nanti dicari

In [ ]:
data.Card_grade.value_counts()*100/data.shape[0]

## Questionnaire

In [ ]:
for i in range(1,6):
    print(data['questionnaire'+str(i)].value_counts())
    print('')
#this result I think quetionnaire doesn't give good information because majority of people doesn't fill the quetionnaire.Ask the client if the client have large number asnswered questionnaire

## Util

In [ ]:
charge=[i for i in data.columns if 'charge_' in i]
charge.remove('charge_avg_per_mon')
charge.remove('charge_lmon')


In [ ]:
data[charge]=data[charge].fillna(0)

In [ ]:
data['charge_weekday']=data['charge_Monday']+data['charge_Tuesday']+data['charge_Wednesday']+data['charge_Thursday']+data['charge_Friday']
data['charge_weekend']=data['charge_Saturday']+data['charge_Sunday']

In [ ]:
data.drop(charge,axis=1,inplace=True)

# Bivariate

## Makes New features based on shoping,cashing,ETC

In [ ]:
#card function column

def card_func(row):
    if row['Shopping']==1 and row['Cashing']==0  and row['ETC']==0:
        return 'S'
    elif row['Shopping']==0 and row['Cashing']==1  and row['ETC']==0:
        return 'C'
    elif row['Shopping']==0 and row['Cashing']==0  and row['ETC']==1:
        return 'E'
    elif row['Shopping']==1 and row['Cashing']==1  and row['ETC']==0:
        return 'SC'
    elif row['Shopping']==1 and row['Cashing']==0  and row['ETC']==1:
        return 'SE'
    elif row['Shopping']==0 and row['Cashing']==1  and row['ETC']==1:
        return 'CE'
    elif row['Shopping']==1 and row['Cashing']==1  and row['ETC']==1:
        return 'SCE'
    elif row['Shopping']==0 and row['Cashing']==0  and row['ETC']==0:
        return 'none' #card doesn't have any function

data['card_func']=data.apply(card_func,axis=1)


## Card_contraction & Expiration Date

In [ ]:
#for bivariate process
data['Card_contraction']=pd.to_datetime(data.Card_contraction)
data['expiration_date']=pd.to_datetime(data.expiration_date)
data['card_duration(years)']=(data['expiration_date']-data['Card_contraction']).dt.days/365  #make new variable card duraiton

## Card_contraction & contraction_year,contraction_mon,contraction_day

In [ ]:
#for bivairate process
data['contraction_year']=data.Card_contraction.dt.year
data['contraction_mon']=data.Card_contraction.dt.month
data['contraction_day_JP']=data.Card_contraction.dt.day_name()


## Age & generations

In [ ]:
#for bivariate process
data[data.Age>100][['Age','generation']]

In [ ]:
data[(data.Age.isna())&(~data.generation.isna())][['Age','generation']]

In [ ]:
#cleaning age based on generations
data.loc[data.Age>100,'Age']=45 #replace with more reasonable value
age_nul=data[(data.Age.isna())&(~data.generation.isna())][['Age','generation']] #fill nan value with generation column
age_nul['Age_new']=age_nul['generation'].str[:2].astype(int)+5
data.loc[(data.Age.isna())&(~data.generation.isna()),['Age']]=age_nul['Age_new']

In [ ]:
#cleaning generations based on age
def generation_new(age):
        try:
            new_gen=str(int(np.floor(age / 10) * 10))+'s'
            return new_gen
        except:
            return np.nan
data['generation']=data['Age'].apply(generation_new)

In [ ]:
data[['generation','Age']].isna().sum()/data.shape[0]

# Multivariate Analysis

## Drop column

In [ ]:
data.dropna(subset='Response', inplace=True) #drop row with null response value

In [ ]:
#drop questionnaire
data.drop(['questionnaire1','questionnaire2','questionnaire3','questionnaire4','questionnaire5'],axis=1,inplace=True)  #drop quotinaire

In [ ]:
missing_values = (data.isnull().sum()/len(data)*100).reset_index()
column_to_drop = list(missing_values[missing_values[0] >= 60]['index'].unique()) #drop nan value more than 70 %

data.drop(column_to_drop, axis=1, inplace=True)

In [ ]:
#data.drop(['Shopping','Cashing','ETC'],axis=1,inplace=True) #drop shopping ,cashing,ETC
data.drop('contraction_day',axis=1,inplace=True)   #drop contraction day

In [ ]:
usage_avg_mon = [i for i in data.columns if 'avg_mon' in i]
usage_avg_lmon = [i for i in data.columns if 'avg_lmon' in i]

In [ ]:
usage_avg_lmon

In [ ]:
#drop date colomn
date_feat = data.select_dtypes('datetime64').columns.tolist() + ['max_used_date', 'min_used_date']
data.drop(date_feat,axis=1,inplace=True)

In [ ]:
#Drop age because with generation, we will make personal more easy
data.drop('generation', axis=1, inplace=True)

In [ ]:
(data.isnull().sum()/len(data)*100).sort_values(ascending=False)

In [ ]:
data.drop_duplicates(subset=['Customer_ID'],inplace=True)

In [ ]:
data.drop(['Customer_ID','NO'],axis=1,inplace=True)

In [ ]:
charge=[i for i in data.columns if 'charge_' in i]
charge

## FIll nan value

In [ ]:
(data.isnull().sum()/len(data)*100).sort_values(ascending=False)

### numeric

In [ ]:
numeric=['ETC_avg','ETC_rec','travel_avg_lmon','travel_avg_mon','speciality_avg_lmon','speciality_avg_mon','cashing_avg_mon','cashing_avg_lmon','cashing_avg','cashing_rec','shopping_rec','shopping_avg']

In [ ]:
for i in numeric:
  data[i].fillna(data[i].mean(), inplace=True)

In [ ]:
data[numeric].isna().sum()

### categorical

In [ ]:
#Change general payer to the same word
def job(row):
  if row in ['general payer1','general payer2','general payer3']:
    x = 'general payer'
  else:
    x = row
  return x

data['Job'] = data['Job'].apply(job)
data['Job'].fillna(data['Job'].mode()[0], inplace=True)

In [ ]:
data['marriage'].fillna(data['marriage'].mode()[0], inplace=True)

## Last Drop

In [ ]:
data.info()

In [ ]:
data.drop(['charge_weekend','charge_weekday','On_the_job'],axis=1,inplace=True)

In [ ]:
data.drop(list(data.columns[18:32])+list(data.columns[37:43])+['On_the_job','card_func','Card_grade','No_records',' No_member_store_use'],axis=1,inplace=True)

## One Hot Encoding

In [ ]:
data.info()

In [ ]:
# convert all columns to string except float 64
cols_to_convert = data.columns[data.dtypes == 'object'].tolist()+['Shopping','Cashing','ETC']
data[cols_to_convert] = data[cols_to_convert].astype(str)

In [ ]:
cols_to_convert =  data.columns[data.dtypes=='object'].tolist()
cols_to_convert.remove('Response')

encoded_data = pd.get_dummies(data[cols_to_convert])


In [ ]:
cols_not_convert =  data.columns[data.dtypes!='object'].tolist()

In [ ]:
cols_not_convert

In [ ]:
features=pd.concat([encoded_data,data[cols_not_convert]],axis=1)

## Tree Visualization

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Y = data['Response']
X = features

dt = DecisionTreeClassifier(random_state=42, max_depth = 3)
dt.fit(X,Y)

In [ ]:
Y.value_counts()

In [ ]:
importances = pd.DataFrame({'feature': X.columns, 'importance': dt.feature_importances_})

# Sort the dataframe by feature importances in descending order
importances = importances.sort_values('importance', ascending=False)

# Print the table of feature importances
print(importances)


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

dot_data = export_graphviz(dt, feature_names=X.columns,  class_names=np.array(['no reply','reply']),filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph